In [1]:
!pip install onnx onnxruntime skl2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.2 which is incompatible.
tensorflow 2

### Import Libraries

In [21]:
import time
import os
import pickle
import joblib
import onnx
import numpy as np
import torch
import onnxruntime as ort
import tensorflow as tf
import pandas as pd
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Random Forest Model

In [12]:
# Load dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)

# Train a model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [13]:
# Function to measure serialization and inference time
def measure_serialization(save_func, load_func, filename, model, inference_data):
    # Measure save time
    start = time.time()
    save_func(model, filename)
    save_time = time.time() - start

    # Measure file size
    file_size = os.path.getsize(filename)

    # Measure load time
    start = time.time()
    loaded_model = load_func(filename)
    load_time = time.time() - start

    # Measure inference time
    start = time.time()
    if isinstance(loaded_model, tf.keras.Model):  # TensorFlow
        _ = loaded_model.predict(inference_data)
    elif isinstance(loaded_model, ort.InferenceSession):  # ONNX
        inputs = {loaded_model.get_inputs()[0].name: inference_data.astype(np.float32)}
        _ = loaded_model.run(None, inputs)
    else:  # Scikit-learn
        _ = loaded_model.predict(inference_data)
    inference_time = time.time() - start

    return save_time, load_time, file_size, inference_time

os.makedirs("models", exist_ok=True)

### Pickle

In [14]:
pickle_file = "models/model.pkl"
pickle_save = lambda model, filename: pickle.dump(model, open(filename, "wb"))
pickle_load = lambda filename: pickle.load(open(filename, "rb"))
pickle_times = measure_serialization(pickle_save, pickle_load, pickle_file, model, X_test)

### Joblib

In [15]:
joblib_file = "models/model.joblib"
joblib_save = lambda model, filename: joblib.dump(model, filename)
joblib_load = lambda filename: joblib.load(filename)
joblib_times = measure_serialization(joblib_save, joblib_load, joblib_file, model, X_test)

### ONNX

In [18]:
onnx_file = "models/model.onnx"
def save_onnx(model, filename):
    onnx_model = convert_sklearn(
        model,
        initial_types=[("input", FloatTensorType([None, X_train.shape[1]]))]
    )
    with open(filename, "wb") as f:
        f.write(onnx_model.SerializeToString())

def load_onnx(filename):
    return ort.InferenceSession(filename)

save_onnx(model, onnx_file)
onnx_times = measure_serialization(save_onnx, load_onnx, onnx_file, model, X_test)

### Tensorflow

In [19]:
tf_file = "models/model_tf.h5"
def save_tf(model, filename):
    tf_model = tf.keras.Sequential([tf.keras.layers.Input(shape=(X_train.shape[1],))])
    tf_model.add(tf.keras.layers.Dense(units=20, activation="relu"))
    tf_model.add(tf.keras.layers.Dense(units=3, activation="softmax"))
    tf_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
    tf_model.save(filename)

def load_tf(filename):
    return tf.keras.models.load_model(filename)

save_tf(model, tf_file)
tf_times = measure_serialization(lambda m, f: save_tf(m, f), load_tf, tf_file, model, X_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


### Results

In [22]:
# Summarize results
results = pd.DataFrame({
    "Method": ["Pickle", "Joblib", "ONNX", "TensorFlow"],
    "Save Time (s)": [pickle_times[0], joblib_times[0], onnx_times[0], tf_times[0]],
    "Load Time (s)": [pickle_times[1], joblib_times[1], onnx_times[1], tf_times[1]],
    "File Size (bytes)": [pickle_times[2], joblib_times[2], onnx_times[2], tf_times[2]],
    "Inference Time (s)": [pickle_times[3], joblib_times[3], onnx_times[3], tf_times[3]],
})

# Display results
print(results)

       Method  Save Time (s)  Load Time (s)  File Size (bytes)  \
0      Pickle       0.007815       0.002852             174046   
1      Joblib       0.064917       0.029788             183761   
2        ONNX       0.063148       0.003013              78502   
3  TensorFlow       0.091890       0.136838              20056   

   Inference Time (s)  
0            0.005002  
1            0.004713  
2            0.000641  
3            0.576529  
